In [ ]:
import numpy as np
import pickle
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image





In [ ]:
#Here we have called vgg16 for its implementation before extraction .
# Got to know about it from this blog
# link:- https://franky07724-57962.medium.com/using-keras-pre-trained-models-for-feature-extraction-in-image-clustering-a142c6cdf5b1#:~:text=Using%20Keras'%20Pre%2Dtrained%20Models%20for%20Feature%20Extraction%20in%20Image%20Clustering,-franky&text=Keras%20provides%20a%20set%20of,feature%20extraction%2C%20and%20transfer%20learning.

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(220, 220, 3)) # Using 3 for dimensional aspect


In [ ]:
#References used to work with vgg16
#We used img_to_array func from keras library, becuase there was a shape issue happening during extraction
#link:- https://www.tensorflow.org/api_docs/python/tf/keras/utils/img_to_array
#Used this blog for code and pre-processing implementation
#link: https://franky07724-57962.medium.com/using-keras-pre-trained-models-for-feature-extraction-in-image-clustering-a142c6cdf5b1#:~:text=Using%20Keras'%20Pre%2Dtrained%20Models%20for%20Feature%20Extraction%20in%20Image%20Clustering,-franky&text=Keras%20provides%20a%20set%20of,feature%20extraction%2C%20and%20transfer%20learning.

def convo_features(img_path):
    img = image.load_img(img_path, target_size=(220, 220))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = base_model.predict(x)
    return features.flatten()



In [ ]:
import pandas as pd
train_csv = pd.read_csv('/kaggle/input/human-action-recognition-har-dataset/Human Action Recognition/Training_set.csv')
test_csv = pd.read_csv('/kaggle/input/human-action-recognition-har-dataset/Human Action Recognition/Testing_set.csv')
train_path='/kaggle/input/human-action-recognition-har-dataset/Human Action Recognition/train'
test_path='/kaggle/input/human-action-recognition-har-dataset/Human Action Recognition/test/'


In [ ]:
all_img_addr=[]
all_label_addr=[]

# Created 2 arrays using containing all image path and their mapped labels
for i in range(0, train_csv.shape[0]):
        img_path = train_path +'/'+ train_csv['filename'][i]
        label_path= train_csv['label'][i]
        all_img_addr.append(img_path)
        all_label_addr.append(label_path)
        
print(len(all_img_addr))
print(len(all_label_addr))

In [ ]:
all_img_addr[0]

In [ ]:

convo_feat = all_img_addr
final_labels = all_label_addr



In [ ]:
#Note:- Main feature extraction is happening here, every image addr is going insider convo_features_func() and featires are getting extracted
temp_convo_features = [convo_features(i) for i in convo_feat]

# converting to numpy arrays, as numpy arrays are accpetable in SVM's
numpy_convo_features = np.array(temp_convo_features)

In [ ]:

# Splitting dataset and validating training sets creation
main_features, main_test_feat, main_train_label, main_test_label = train_test_split(numpy_convo_features, final_labels, test_size=0.2, random_state=42)



In [ ]:
print(numpy_convo_features[0])

In [ ]:
# Reducing Dimensions using PCA at saving 95% varianve
pca = PCA(n_components=0.95)
train_pca = pca.fit_transform(main_features)
test_pca = pca.transform(main_test_feat)


In [ ]:
C_val=[0.1,1]
kernel_val=['rbf','linear']

# Called and initiated SVM classifier
svm_convo_model = SVC(C=C_val[1], kernel=kernel_val[0]) # keeping gamma default as scaled gamma was giving low accuracy


In [ ]:
svm_convo_model.fit(train_pca, main_train_label)  # Training model by fitting training data got after spllting data

# checking trained model using splitted test_data
accuracy = svm_convo_model.score(test_pca, main_test_label)
print(f'Accuracy for linear SVM: {accuracy}')

In [ ]:
#Note:- this method to save model and download it for further use, i learnt it using chatgpt as other methods were giving some error
with open('svm_model_rbf_dummy.pkl', 'wb') as svm_model_file:
    pickle.dump(svm_convo_model, svm_model_file)

In [ ]:

with open('pca_model_1.pkl', 'wb') as pca_model_file:
    pickle.dump(pca, pca_model_file)

In [ ]:

extracted_feature_tester=convo_features('/kaggle/input/human-action-recognition-har-dataset/Human Action Recognition/train/Image_1.jpg')
